In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

import matplotlib.pyplot as plt

from scipy import stats
import math
import random

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

from xgboost import XGBRegressor

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test['SalePrice'] = 0
df_concat = pd.concat([df_train,df_test])

In [4]:
def giveMeWrangledData(df, testFile=False):
    
    
    df = df.drop(['Id', 'GarageYrBlt','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF'],axis=1)
    
    df['LotFrontage'] =df.LotFrontage.fillna(df.LotFrontage.mode()[0])
    df['MasVnrArea']=df.MasVnrArea.fillna(0.0)
    df['TotalBsmtSF'] = df.TotalBsmtSF.fillna(0)
    df['BsmtFullBath'] = df.BsmtFullBath.fillna(0)
    df['BsmtHalfBath'] = df.BsmtHalfBath.fillna(0)
    df['GarageCars'] = df.GarageCars.fillna(0)
    df['GarageArea'] = df.GarageArea.fillna(0)
    
    #convert data type
    #we are being little lineant to give int64 for YearBuilt, YrSold but those guys are going to be box-coxed 
    #so let them at least enjoy the bigger size for now
    int64_variables = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', \
                     'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', \
                     'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',\
                     'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', \
                     'PoolArea', 'MiscVal', 'YrSold', 'SalePrice']
    
    #if testFile:
    #    int64_variables.remove('SalePrice')

        
        
        
    
    for c in int64_variables:
        print("Changing the data type for :", c)
        df[c] = df[c].astype(np.int64)
        
    int_to_categorical_variables = ['MSSubClass', 'OverallQual', 'OverallCond', 'FireplaceQu', 'MoSold']
    for c in int_to_categorical_variables:
        df[c] = df[c].astype(str)
        
    df = df.fillna('NotAvailable')
    return df
df = giveMeWrangledData(df_concat)
df.head()

Changing the data type for : LotFrontage
Changing the data type for : LotArea
Changing the data type for : YearBuilt
Changing the data type for : YearRemodAdd
Changing the data type for : MasVnrArea
Changing the data type for : TotalBsmtSF
Changing the data type for : 1stFlrSF
Changing the data type for : 2ndFlrSF
Changing the data type for : LowQualFinSF
Changing the data type for : GrLivArea
Changing the data type for : BsmtFullBath
Changing the data type for : BsmtHalfBath
Changing the data type for : FullBath
Changing the data type for : HalfBath
Changing the data type for : BedroomAbvGr
Changing the data type for : KitchenAbvGr
Changing the data type for : TotRmsAbvGrd
Changing the data type for : Fireplaces
Changing the data type for : GarageCars
Changing the data type for : GarageArea
Changing the data type for : WoodDeckSF
Changing the data type for : OpenPorchSF
Changing the data type for : EnclosedPorch
Changing the data type for : 3SsnPorch
Changing the data type for : Scree

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65,8450,Pave,NotAvailable,Reg,Lvl,AllPub,Inside,...,0,NotAvailable,NotAvailable,NotAvailable,0,2,2008,WD,Normal,208500
1,20,RL,80,9600,Pave,NotAvailable,Reg,Lvl,AllPub,FR2,...,0,NotAvailable,NotAvailable,NotAvailable,0,5,2007,WD,Normal,181500
2,60,RL,68,11250,Pave,NotAvailable,IR1,Lvl,AllPub,Inside,...,0,NotAvailable,NotAvailable,NotAvailable,0,9,2008,WD,Normal,223500
3,70,RL,60,9550,Pave,NotAvailable,IR1,Lvl,AllPub,Corner,...,0,NotAvailable,NotAvailable,NotAvailable,0,2,2006,WD,Abnorml,140000
4,60,RL,84,14260,Pave,NotAvailable,IR1,Lvl,AllPub,FR2,...,0,NotAvailable,NotAvailable,NotAvailable,0,12,2008,WD,Normal,250000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 76 columns):
MSSubClass       2919 non-null object
MSZoning         2919 non-null object
LotFrontage      2919 non-null int64
LotArea          2919 non-null int64
Street           2919 non-null object
Alley            2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2919 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null object
OverallCond      2919 non-null object
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 n

In [6]:
def preProcessData(df):
    
    print("Shape of the data set before pre processing : ", df.shape )

    
    #get dummies
    print("Categorical columns : ", list(df.select_dtypes(exclude=np.number)))
    df = pd.get_dummies(df)
    #df = df.drop(categorical_columns, axis=1)
    
    print("\n\nShape of the data set after pre processing : ", df.shape )
    
    print("Columns in the data set are : ",list(df))

    return df
df_prep = preProcessData(df)
df_prep.info()

Shape of the data set before pre processing :  (2919, 76)
Categorical columns :  ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition']


Shape of the data set after pre processing :  (2919, 351)
Columns in the data set are :  ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 

In [27]:
def newBoxCoxTranformation(df,target,testFile=False):
    
    #assuming that only numerical features are presented
    print("Shape of the dataset initial : ", df.shape)
    
    if not testFile:
        df =df[df.SalePrice >0]
        print("Shape of the dataset before transformation : ", df.shape)
        y = np.array(df[target].apply( lambda x: math.log(x)))
        X= df.drop(target,axis = 1)
        #x_columns = list(X)
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        X = preprocessing.power_transform( X, method='box-cox')
        #X = pd.DataFrame(X,columns=x_columns)
        print("Shape of the dataset after transformation : ", X.shape, y.shape)
        return X,y
    else:
        df = df[df.SalePrice == 0.0]
        print("Shape of the dataset before transformation : ", df.shape)
        X=df.drop(target,axis = 1)
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        X = preprocessing.power_transform( X, method='box-cox')
        print("Shape of the dataset after transformation : ", X.shape)
        return X
        
    

    
X,y = newBoxCoxTranformation(df_prep,'SalePrice')

Shape of the dataset initial :  (2919, 351)
Shape of the dataset before transformation :  (1460, 351)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


Shape of the dataset after transformation :  (1460, 350) (1460,)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=random.randint(1,500))#, stratify=df.BldgType)

In [29]:
reg = XGBRegressor()
reg.fit(X_train,y_train)
reg.score(X_test,y_test)

0.882673075590771

In [30]:
reg

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [31]:
np.sqrt(mean_squared_log_error(y_test, reg.predict(X_test)))

0.010664576654458021

##### We need to have different pre-processing logic to test data. We will come back to it little later.

In [36]:
def checkTheTestFile():
    df_test = pd.read_csv('test.csv')
    df_test['SalePrice'] = 0.0
    
    df_train =  pd.read_csv('train.csv')
    df_concat = pd.concat([df_train,df_test])

    #print(df_test[df_test.TotalBsmtSF.isna()])
    #return
    df = giveMeWrangledData(df_concat,True)
    
    print(df.info())
    df = preProcessData(df)
    print(df.info())
    X = newBoxCoxTranformation(df,'SalePrice',True)
    #print(np.sqrt(mean_squared_log_error(y, reg.predict(X))))
    
    df_test['SalePrice'] = np.exp(reg.predict(X))
    return df_test
df_test = checkTheTestFile()

Changing the data type for : LotFrontage
Changing the data type for : LotArea
Changing the data type for : YearBuilt
Changing the data type for : YearRemodAdd
Changing the data type for : MasVnrArea
Changing the data type for : TotalBsmtSF
Changing the data type for : 1stFlrSF
Changing the data type for : 2ndFlrSF
Changing the data type for : LowQualFinSF
Changing the data type for : GrLivArea
Changing the data type for : BsmtFullBath
Changing the data type for : BsmtHalfBath
Changing the data type for : FullBath
Changing the data type for : HalfBath
Changing the data type for : BedroomAbvGr
Changing the data type for : KitchenAbvGr
Changing the data type for : TotRmsAbvGrd
Changing the data type for : Fireplaces
Changing the data type for : GarageCars
Changing the data type for : GarageArea
Changing the data type for : WoodDeckSF
Changing the data type for : OpenPorchSF
Changing the data type for : EnclosedPorch
Changing the data type for : 3SsnPorch
Changing the data type for : Scree

C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:900: RuntimeWarning: divide by zero encountered in log
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


Shape of the dataset after transformation :  (1459, 350)


C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [37]:
df_test[['Id','SalePrice']]

,Id,SalePrice
0,1461,128009.539062
1,1462,157759.843750
2,1463,189456.656250
3,1464,190033.546875
4,1465,182320.359375
5,1466,183233.734375
6,1467,170539.375000
7,1468,171253.546875
8,1469,191111.093750
9,1470,121259.656250
